In [1]:
import sys
import uproot

import os
import numpy as np
import pandas as pd


In [2]:
part = "pion_c" # "gamma" "muon" "electron"
path = "/data/user/adipilat/ParticleID/genEvts/"
dir_ = "ana"
tree = "hgc"
max_perlayer = 10
number_layers = 50

In [3]:
variableName = [
            'event','run','lumi',
            'cluster2d_layer',
            'cluster2d_energy',
            'cluster2d_eta',
            'cluster2d_phi',
            'cluster2d_pt',
            'cluster2d_x',
            'cluster2d_y',
            'cluster2d_z',
            'cluster2d_nhitCore',
            'cluster2d_nhitAll',
            'gen_energy',
            'gen_pdgid',
            'gen_daughters',
            'gen_phi',
            'gen_eta',
            'trackster_clusters'
            ]
newVars =["run","lumi","event","trackster","x","y","z","r","layer","E","nCore","nHits","id","genDR","gen_phi","gen_eta","phi","eta","pid","genE"]

In [4]:
print("Starting data production")
name = "4_" + part
file = path + part + "/" + name + ".root"

Starting data production


In [5]:
df = uproot.open(file)[dir_][tree].pandas.df(variableName,flatten=False)


num_events = np.unique(df["event"].values).shape[0]
xs = df["cluster2d_x"].values
ys = df["cluster2d_y"].values
zs = df["cluster2d_z"].values
es = df["cluster2d_energy"].values
ps = df["cluster2d_pt"].values
nh = df["cluster2d_nhitAll"].values
nc = df["cluster2d_nhitCore"].values
ll = df["cluster2d_layer"].values
ee = df["event"].values
lu = df["lumi"].values
ru = df["run"].values
    
sizes = [x.shape[0] for x in xs]
indices = [np.full((a[0]),a[1]) for a in zip(sizes,range(len(sizes)))]
        #gendau = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_daughters"].values)]

cphi = df["cluster2d_phi"].values
ceta = df["cluster2d_eta"].values
gpid = df["gen_pdgid"].values
gen = df["gen_energy"].values

gphi = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_phi"].values)]
geta = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_eta"].values)]
gpid = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_pdgid"].values)]
gen = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_energy"].values)]


tr = df["trackster_clusters"].values

In [6]:
idtrlist = []
# LayerClusters that don't belong to any Trackster will have TracksterId = 0. Real Tracksters have the TracksterId > 0
for i in range(len(sizes)):
    idtrlist.append(np.array([0]*sizes[i]))
for i in range(len(tr)):
    for j in range(len(tr[i])):
        for item in tr[i][j]:
            idtrlist[i][item] = j + 1

idtr = np.array(idtrlist)

In [7]:
rs = [np.sqrt(f[0]**2+f[1]**2) for f in zip(xs,ys)]

drs = [np.sqrt((a[0]-a[1])**2 + (a[2]-a[3])**2) for a in zip(gphi,cphi,geta,ceta)]


In [8]:
XS = np.array([item for sublist in xs for item in sublist])
YS = np.array([item for sublist in ys for item in sublist])
ZS = np.array([item for sublist in zs for item in sublist])
RS = np.array([item for sublist in rs for item in sublist])
LL = np.array([item for sublist in ll for item in sublist])
ES = np.array([item for sublist in es for item in sublist])
NC = np.array([item for sublist in nc for item in sublist])
NH = np.array([item for sublist in nh for item in sublist])
II = np.array([item for sublist in indices for item in sublist])
DRS = np.array([item for sublist in drs for item in sublist])
GPHI = np.array([item for sublist in gphi for item in sublist])
GETA = np.array([item for sublist in geta for item in sublist])
GPID = np.array([item for sublist in gpid for item in sublist])
GEN = np.array([item for sublist in gen for item in sublist])
CPHI = np.array([item for sublist in cphi for item in sublist])
CETA = np.array([item for sublist in ceta for item in sublist])


SS = [np.full((s,),s) for s in sizes]
EE = [np.full((s,),i) for i,s in zip(ee,sizes)]
LU = [np.full((s,),i) for i,s in zip(lu,sizes)]
RU = [np.full((s,),i) for i,s in zip(ru,sizes)]


SS = np.array([item for sublist in SS for item in sublist])
EE = np.array([item for sublist in EE for item in sublist])
LU = np.array([item for sublist in LU for item in sublist])
RU = np.array([item for sublist in RU for item in sublist])

TR = np.array([item for sublist in idtr for item in sublist])

datas = np.vstack((RU,LU,EE,TR,XS,YS,ZS,RS,LL,ES,NC,NH,II,DRS,GPHI,GETA,CPHI,CETA,GPID,GEN)).T

In [9]:
df = pd.DataFrame(datas,columns=newVars)
df = df.sort_values(["event","trackster","layer","E"],ascending=[True,True,True,False]).reset_index(drop=True)

In [10]:
df.to_hdf(path + part + ".h5","data",complevel=0)

In [11]:
df[80:150]

,run,lumi,event,trackster,x,y,z,r,layer,E,nCore,nHits,id,genDR,gen_phi,gen_eta,phi,eta,pid,genE
80,1.0,1.0,1.0,0.0,80.732040,-36.425205,339.922760,88.568939,13.0,0.083580,1.0,1.0,23.0,0.104393,-0.339637,2.116348,-0.423840,2.054640,211.0,367.569397
81,1.0,1.0,1.0,0.0,70.808243,-65.229767,339.922760,96.274239,13.0,0.074740,2.0,2.0,23.0,0.429031,-0.339637,2.116348,-0.744414,1.974141,211.0,367.569397
82,1.0,1.0,1.0,0.0,88.107574,-13.983215,339.922760,89.210289,13.0,0.068712,1.0,1.0,23.0,0.194758,-0.339637,2.116348,-0.157393,2.047660,211.0,367.569397
83,1.0,1.0,1.0,0.0,72.027458,-34.008377,339.922760,79.652519,13.0,0.045147,1.0,1.0,23.0,0.109571,-0.339637,2.116348,-0.441127,2.157645,211.0,367.569397
84,1.0,1.0,1.0,0.0,90.200600,-46.783211,339.922760,101.611107,13.0,0.042192,1.0,1.0,23.0,0.238562,-0.339637,2.116348,-0.478462,1.922338,211.0,367.569397
85,1.0,1.0,1.0,0.0,85.964569,-22.528603,339.922760,88.867569,13.0,0.040986,1.0,1.0,23.0,0.105663,-0.339637,2.116348,-0.256305,2.051383,211.0,367.569397
86,1.0,1.0,1.0,0.0,77.542542,-37.116009,339.922760,85.967690,13.0,0.028932,1.0,1.0,23.0,0.111721,-0.339637,2.116348,-0.446425,2.083514,211.0,367.569397
87,1.0,1.0,1.0,0.0,75.449516,-35.907604,339.922760,83.558273,13.0,0.020091,1.0,1.0,23.0,0.104690,-0.339637,2.116348,-0.444195,2.111096,211.0,367.569397
88,1.0,1.0,1.0,0.0,83.871567,-30.987402,339.922760,89.412857,13.0,0.018082,1.0,1.0,23.0,0.072303,-0.339637,2.116348,-0.353907,2.045466,211.0,367.569397
89,1.0,1.0,1.0,0.0,65.980980,-55.932804,339.922760,86.498375,13.0,0.015269,1.0,1.0,23.0,0.365591,-0.339637,2.116348,-0.703164,2.077549,211.0,367.569397
